In [1]:
# function calling with openai

In [2]:
import os
import json
import pandas as pd

import openai
from openai import OpenAI

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

import pdb

import dotenv
dotenv.load_dotenv()

True

In [3]:
openai.api_key = os.getenv('OPENAI_API_KEY')
MODEL='gpt-4-turbo'

spotify_id = os.getenv('SPOTIFY_CLIENT_ID')
spotify_secret = os.getenv('SPOTIFY_CLIENT_SECRET')

# models = openai.Model.list()
# print([(i, m.id,) for i, m in enumerate(models["data"])])
# models['data'][2]

In [4]:
auth_manager = SpotifyClientCredentials(client_id=spotify_id, client_secret=spotify_secret)
spotify = spotipy.Spotify(auth_manager=auth_manager)


In [5]:
openai_client = OpenAI()


In [6]:
# tool to get playlist id
def get_playlist_id(playlist_name, spotify=spotify, verbose=False):
    """
    Retrieves the playlist ID from Spotify based on the playlist name.

    Args:
        playlist_name (str): The name of the playlist to search for.
        spotify (object): The Spotify object used for API calls (default: spotify).
        verbose (bool): If True, prints additional information about the playlist (default: False).

    Returns:
        str or None: The ID of the playlist if found, None otherwise.
    """
    playlists = spotify.user_playlists(os.getenv('SPOTIFY_USERNAME'))
    while playlists:
        for i, playlist in enumerate(playlists['items']):
            if playlist['name'] == playlist_name:
                if verbose:
                    print('"%s": offset %d, URI %s' % (playlist['name'], i + 1 + playlists['offset'], playlist['uri']))
                return playlist['id']

        # not found yet, get next page if there is one
        if playlists['next']:
            playlists = sp.next(playlists)
        else:
            return None

playlist_id = get_playlist_id("swing")
print(playlist_id)

5U7jjAdmClwguFUUrF8DtF


In [7]:
# tool to get a JSON list of playlist tracks based on playlist id
def get_playlist_tracks(playlist_id, spotify=spotify, verbose=False):
    """
    Retrieves tracks from a Spotify playlist based on the provided playlist ID.

    Args:
        playlist_id (str): The ID of the Spotify playlist.
        spotify (object, optional): The Spotify API object. Defaults to the global `spotify` object.
        verbose (bool, optional): If True, additional information will be printed during execution. Defaults to False.

    Returns:
        pandas.DataFrame: A DataFrame containing information about the tracks in the playlist. The DataFrame has the following columns:
            - artist: The name of the artist.
            - track: The name of the track.
            - uri: The URI of the track.
            - id: The ID of the track.
            - popularity: The popularity score of the track.
    """
    tracks = []
    results = spotify.playlist_items(playlist_id)
    if results:
        tracks = results['items']
        while results['next']:
            # print('.', end='')
            tracks.extend(results['items'])
            results = spotify.next(results)

    retlist = [{'artist': track['track']['artists'][0]['name'],
                'track': track['track']['name'],
                'uri': track['track']['uri'],
                'id': track['track']['id'],
                'popularity': track['track']['popularity']} for track in tracks]

    return json.dumps(retlist)

tracks = get_playlist_tracks(playlist_id)
tracks


'[{"artist": "Benny Goodman", "track": "King Porter Stomp", "uri": "spotify:track:7t1k8CsFnFonvI9b2zoRsZ", "id": "7t1k8CsFnFonvI9b2zoRsZ", "popularity": 0}, {"artist": "Cherry Poppin\' Daddies", "track": "No Mercy for Swine", "uri": "spotify:track:3td6JTzJXVbkkGET2qKuaj", "id": "3td6JTzJXVbkkGET2qKuaj", "popularity": 22}, {"artist": "Louis Prima", "track": "Jump, Jive, An\' Wail - Remastered", "uri": "spotify:track:2JknWUrnGsGYOh62EQNktb", "id": "2JknWUrnGsGYOh62EQNktb", "popularity": 43}, {"artist": "Squirrel Nut Zippers", "track": "Hell - Remastered 2016", "uri": "spotify:track:4efAv86uRxR4yQBcb3Vczq", "id": "4efAv86uRxR4yQBcb3Vczq", "popularity": 44}, {"artist": "Squirrel Nut Zippers", "track": "Put a Lid on It - Remastered 2016", "uri": "spotify:track:4OHXGwmWLzb6Vdcnc0nBfK", "id": "4OHXGwmWLzb6Vdcnc0nBfK", "popularity": 37}, {"artist": "Squirrel Nut Zippers", "track": "Suits Are Picking Up The Bill", "uri": "spotify:track:6HKwawGbOk1sURnB7Xwat1", "id": "6HKwawGbOk1sURnB7Xwat1", "p

In [9]:
# set up tools for an openai call
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_playlist_id",
            "description": "Return the Spotify playlist id based on the name of the playlist",
            "parameters": {
                "type": "object",
                "properties": {
                    "playlist_name": {
                        "type": "string",
                        "description": "The name of the playlist"
                    },
                },
                "required": ["playlist_name"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_playlist_tracks",
            "description": "Return a list of the Spotify tracks in a playlist based on the playlist id",
            "parameters": {
                "type": "object",
                "properties": {
                    "playlist_id": {
                        "type": "string",
                        "description": "The id of the playlist"
                    },
                },
                "required": ["playlist_id"],
            },
        },
    },
]

# map tool names to local functions
available_functions = {
        "get_playlist_id": get_playlist_id,
        "get_playlist_tracks": get_playlist_tracks,
    }


In [8]:
system_prompt = """You are a Spotify helper AI.
I will make requests related to Spotify and you will answer using the functions provided"
"""


In [10]:
def eval_tool(tool_call):
    """
    Given an OpenAI tool_call response,
    evaluates the tool function using the arguments provided by OpenAI,
    and returns the message to send back to OpenAI, including the function return value.

    Args:
        tool_call (object): The OpenAI tool_call response.

    Returns:
        dict: The message to send back to OpenAI, containing the tool_call_id, role, name, and value returned by the tool call.

    """
    function_name = tool_call.function.name
    # make json args a dict
    kwargs = json.loads(tool_call.function.arguments)
    fn = available_functions[function_name]
    # call function with the args and return value
    fn_value = fn(**kwargs)
    return {
        "tool_call_id": tool_call.id,
        "role": "tool",
        "name": tool_call.function.name,
        "content": fn_value,
    }


def get_response_json(messages):
    """
    Get a single JSON response from ChatGPT based on a chain of messages.

    Args:
        messages (list): A list of message objects representing the conversation history.

    Returns:
        dict: A response object containing the generated response from ChatGPT.

    Raises:
        OpenAIError: If there is an error during the API call.

    Example:
        >>> messages = [
        ...     {"role": "system", "content": "You are a helpful assistant."},
        ...     {"role": "user", "content": "What's the weather like today?"},
        ... ]
        >>> response = get_response(messages)
    """
    response = openai_client.chat.completions.create(
        model=MODEL,
        messages=messages,
        tools=tools,
        tool_choice="auto",  # auto is default, but we'll be explicit
        response_format={"type": "json_object"},
    )

    return response


def get_response_and_eval_json(messages, verbose=False):
    """
    Sends a list of messages to OpenAI and returns the JSON response.
    If tool calls are returned, calls the tools and sends the values back to OpenAI.
    If additional tool calls returned, iterates until no more tool calls are returned and
    'stop' is returned as finish_reason, then returns the response.

    Args:
        messages (list): A list of messages to send to OpenAI.
        verbose (bool, optional): If True, prints additional information. Defaults to False.

    Returns:
        response: The response object returned by OpenAI.

    Raises:
        None

    """
    response = get_response_json(messages)
    choice = response.choices[0]
    response_message = choice.message
    finish_reason = choice.finish_reason

    if verbose:
        print(choice)

    while finish_reason != 'stop':
        # Extend conversation with assistant's reply
        messages.append(response_message)
        if finish_reason == 'tool_calls':
            tool_calls = response_message.tool_calls
            if verbose:
                print(tool_calls)
            # Call the tools and add all return values as messages
            messages.extend(map(eval_tool, tool_calls))
            # Get next response
            response = get_response_json(messages)
            choice = response.choices[0]
            response_message = choice.message
            finish_reason = choice.finish_reason
            if verbose:
                print(choice)
        else:
            print('finish_reason: ', finish_reason)
            break

    return response


In [11]:
user_message = "Get the id of the playlist entitled 'swing' as a JSON object"
messages = [{"role": "system", "content": system_prompt},
            {"role": "user", "content": user_message}]

response = get_response_and_eval_json(messages, verbose=True)
print(response.choices[0].message.content)


Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_dSyJgxAlCvUtxWj8cZdHSN6e', function=Function(arguments='{"playlist_name":"swing"}', name='get_playlist_id'), type='function')]))
[ChatCompletionMessageToolCall(id='call_dSyJgxAlCvUtxWj8cZdHSN6e', function=Function(arguments='{"playlist_name":"swing"}', name='get_playlist_id'), type='function')]
Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{\n  "playlist_id": "5U7jjAdmClwguFUUrF8DtF"\n}', role='assistant', function_call=None, tool_calls=None))
{
  "playlist_id": "5U7jjAdmClwguFUUrF8DtF"
}


In [12]:
user_message = "Get the list of tracks in my playlist entitled 'swing' as a json object"
messages = [{"role": "system", "content": system_prompt},
            {"role": "user", "content": user_message}]

response = get_response_and_eval_json(messages, verbose=True)
print(response.choices[0].message.content)


Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_rRqiweztZUNwPjXiGGrVO8Tl', function=Function(arguments='{"playlist_name":"swing"}', name='get_playlist_id'), type='function')]))
[ChatCompletionMessageToolCall(id='call_rRqiweztZUNwPjXiGGrVO8Tl', function=Function(arguments='{"playlist_name":"swing"}', name='get_playlist_id'), type='function')]
Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_4cPohPVyqkQBbGVIRvms69iV', function=Function(arguments='{"playlist_id":"5U7jjAdmClwguFUUrF8DtF"}', name='get_playlist_tracks'), type='function')]))
[ChatCompletionMessageToolCall(id='call_4cPohPVyqkQBbGVIRvms69iV', function=Function(arguments='{"playlist_id":"5U7jjAdmClwguFUUrF8DtF"}', name='get_playlist_tracks'), type

In [18]:
pd.DataFrame(json.loads(response.choices[0].message.content)['tracks'])

,artist,track,uri,id,popularity
0,Benny Goodman,King Porter Stomp,spotify:track:7t1k8CsFnFonvI9b2zoRsZ,7t1k8CsFnFonvI9b2zoRsZ,0
1,Cherry Poppin' Daddies,No Mercy for Swine,spotify:track:3td6JTzJXVbkkGET2qKuaj,3td6JTzJXVbkkGET2qKuaj,22
2,Louis Prima,"Jump, Jive, An' Wail - Remastered",spotify:track:2JknWUrnGsGYOh62EQNktb,2JknWUrnGsGYOh62EQNktb,43
3,Squirrel Nut Zippers,Hell - Remastered 2016,spotify:track:4efAv86uRxR4yQBcb3Vczq,4efAv86uRxR4yQBcb3Vczq,44
4,Squirrel Nut Zippers,Put a Lid on It - Remastered 2016,spotify:track:4OHXGwmWLzb6Vdcnc0nBfK,4OHXGwmWLzb6Vdcnc0nBfK,37
5,Squirrel Nut Zippers,Suits Are Picking Up The Bill,spotify:track:6HKwawGbOk1sURnB7Xwat1,6HKwawGbOk1sURnB7Xwat1,31
6,Squirrel Nut Zippers,Good Enough For Granddad,spotify:track:6w4yAE1kA7WDectc4Er0pm,6w4yAE1kA7WDectc4Er0pm,30
7,Cherry Poppin' Daddies,Zoot Suit Riot,spotify:track:1qmJbXpVLydNcN6VTR40GU,1qmJbXpVLydNcN6VTR40GU,50
8,Cherry Poppin' Daddies,The Ding Dong Daddy of the D Car Line,spotify:track:4uxRacg0PBqOIan3lhJ3Xt,4uxRacg0PBqOIan3lhJ3Xt,0
9,Cherry Poppin' Daddies,Master and Slave,spotify:track:6n0IGLnlYkaheqLKHTV1n9,6n0IGLnlYkaheqLKHTV1n9,18


In [20]:
from IPython.display import display, HTML

def print_messages(messages):
    html_output = ""
    for message in messages:
        if type(message) is dict:
            if message['role'] == 'user':
                html_output += f"<div style='margin: 10px; padding: 10px; background-color: lightblue; border-radius: 8px; width: 60%;'><strong>User:</strong> {message['content']}</div>"
            else:
                html_output += f"<div style='margin: 10px; padding: 10px; background-color: lightgreen; border-radius: 8px; width: 60%; float: right; clear: both;'><strong>System:</strong> {message['content']}</div>"
    # else: display tool call
    display(HTML(html_output))


# messages.append(response_message.content)
print_messages(messages)